In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import seaborn as sns

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from io import BytesIO

pd.set_option('display.max_columns', 100)
%matplotlib inline

/opt/conda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# user = []
# ad_operation = []
# imps_log = []
# test_sample = []
# ad_static_feature = []
# with zipfile.ZipFile("data/algo_junior_a.zip") as f:
#   print(f.namelist())
#   for name in f.namelist():
#     if name == 'testA/user.zip':
#       raw_data = BytesIO(f.read(name))
#       with zipfile.ZipFile(raw_data) as f_:
#         user = tf.compat.as_str(f_.read(f_.namelist()[0]))
    
#     if name == 'testA/imps_log.zip':
#       raw_data = BytesIO(f.read(name))
#       with zipfile.ZipFile(raw_data) as f_:
#         imps_log = tf.compat.as_str(f_.read(f_.namelist()[0]))
    
#     if name == 'testA/ad_operation.dat':
#       ad_operation = tf.compat.as_str(f.read(name))
#     if name == 'testA/test_sample.dat':
#       test_sample = tf.compat.as_str(f.read(name))
#     if name == 'testA/ad_static_feature.out':
#       test_sample = tf.compat.as_str(f.read(name))

In [3]:
ad_static_feature = pd.read_csv("data/tencent/testA/ad_static_feature.out",sep='\t',header=None)
ad_operation = pd.read_csv("data/tencent/testA/ad_operation.dat", sep='\t', header=None)
test_sample = pd.read_csv("data/tencent/testA/test_sample.dat", sep='\t', header=None)
user = pd.read_csv("data/tencent/testA/user_data", sep='\t', header=None)
imps_log =  pd.read_csv("data/tencent/testA/totalExposureLog.out", sep='\t', header=None)

/opt/conda/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
user.shape

(1396718, 11)

In [11]:
imps_log.shape

(102386695, 10)

In [12]:
ad_static_feature.shape

(735911, 7)

In [13]:
ad_operation.shape

(760866, 5)

In [14]:
test_sample.shape

(20290, 11)

In [6]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396718 entries, 0 to 1396717
Data columns (total 11 columns):
0     1396718 non-null int64
1     1396718 non-null int64
2     1396718 non-null int64
3     1396718 non-null object
4     1396718 non-null object
5     1396718 non-null int64
6     1396718 non-null int64
7     1396718 non-null int64
8     1396718 non-null object
9     1396718 non-null int64
10    1396718 non-null object
dtypes: int64(7), object(4)
memory usage: 117.2+ MB


In [7]:
imps_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102386695 entries, 0 to 102386694
Data columns (total 10 columns):
0    int64
1    int64
2    int64
3    int64
4    int64
5    int64
6    int64
7    float64
8    float64
9    float64
dtypes: float64(3), int64(7)
memory usage: 7.6 GB


In [8]:
ad_static_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735911 entries, 0 to 735910
Data columns (total 7 columns):
0    735911 non-null int64
1    735911 non-null int64
2    735911 non-null int64
3    735911 non-null object
4    735911 non-null int64
5    735911 non-null object
6    509252 non-null object
dtypes: int64(4), object(3)
memory usage: 39.3+ MB


In [9]:
ad_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760866 entries, 0 to 760865
Data columns (total 5 columns):
0    760866 non-null int64
1    760866 non-null int64
2    760866 non-null int64
3    760866 non-null int64
4    760866 non-null object
dtypes: int64(4), object(1)
memory usage: 29.0+ MB


In [15]:
test_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20290 entries, 0 to 20289
Data columns (total 11 columns):
0     20290 non-null int64
1     20290 non-null int64
2     20290 non-null int64
3     20290 non-null int64
4     20290 non-null int64
5     20290 non-null int64
6     20290 non-null int64
7     20290 non-null int64
8     20290 non-null object
9     20290 non-null object
10    20290 non-null int64
dtypes: int64(9), object(2)
memory usage: 1.7+ MB


In [17]:
!pip install modin

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
    100% |████████████████████████████████| 194kB 437kB/s ta 0:00:011
    100% |████████████████████████████████| 10.1MB 102kB/s eta 0:00:01
    100% |████████████████████████████████| 73.0MB 4.2kB/s eta 0:00:01                     | 8.7MB 32.4MB/s eta 0:00:029MB 28.6MB/s eta 0:00:020MB 32.5MB/s eta 0:00:020:02███████▎                 | 32.7MB 44.3MB/s eta 0:00:01    47% |███████████████▎                | 34.8MB 43.3MB/s eta 0:00:01MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 14.7MB/s ta 0:00:01
  Found existing installation: pandas 0.20.3
    Uninstalling pandas-0.20.3:
      Successfully uninstalled pandas-0.20.3
You are using pip version 18.0, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
